# Train your first PyTorch Model - card classification

1. Pytorch Dtaset
2. Pytorch Model
3. Pytorch Training loop

Almost every pytorch model training pipelline meet this paradigm

# Library

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import timm
import torchvision

from tqdm.notebook import tqdm  

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys 



# Versions

In [ ]:
print('System Version: ', sys.version)
print('PyTorch Version: ', torch.__version__)
print('TorchVision Version', torchvision.__version__)
print('Numpy version',np.__version__)
print('Pandas version',pd.__version__)

# Step 1: Pytorch Dataset and DataLoader
would you learn how to bake a cake

In [ ]:
class PlayingCardDataset(Dataset):
    
    def __init__(self, data_dir, transform=None):
        self.data = ImageFolder(data_dir, transform=transform)
        
    
    def __len__(self):
        return len(self.data)
    
    
    def __getitem__(self,idx):
        return self.data[idx]
    
    @property
    def classses(self):
        return self.data.classes 
    
    
    

In [ ]:
dataset = PlayingCardDataset(
                data_dir='/kaggle/input/cards-image-datasetclassification/train'
)

In [ ]:
len(dataset)

In [ ]:
dataset[5]

In [ ]:
image, label = dataset[5000]
image

In [ ]:
label

In [ ]:
# get a dictionary associating target vvalues with folder  names

data_dir = '/kaggle/input/cards-image-datasetclassification/train'
target_to_class = {v: k for k, v in ImageFolder(data_dir).class_to_idx.items()}
print(target_to_class)


# Setup Datasets

In [ ]:
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(), 
])

data_dir = '/kaggle/input/cards-image-datasetclassification/train'
dataset = PlayingCardDataset(data_dir, transform)

 

In [ ]:
image, label = dataset[100]
image

In [ ]:
image.shape

In [ ]:
label

In [ ]:
## iterate over dataset

for image, label in dataset:
    
    break 

In [ ]:
image, label

# Data loaders
* batching our dataset

In [ ]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [ ]:
for images, labels in dataloader:
    break

In [ ]:
images.shape

In [ ]:
labels.shape

In [ ]:
labels

In [ ]:
transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(), 
])

 

train_folder  = '/kaggle/input/cards-image-datasetclassification/train'
test_folder = '/kaggle/input/cards-image-datasetclassification/test'
val_folder = '/kaggle/input/cards-image-datasetclassification/valid'


train_dataset = PlayingCardDataset(train_folder, transform=transform)
test_dataset = PlayingCardDataset(test_folder, transform=transform)
valid_dataset = PlayingCardDataset(val_folder, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False)

# Step 2: Pytorch Model

* we could create the model from scratch defining each leayer
* TIMM

In [ ]:
class SimpleCardClassifier(nn.Module):
    
    def __init__(self, num_classes=53):
        super(SimpleCardClassifier, self).__init__()
        # where we define all the parts of the model
        self.base_model = timm.create_model('efficientnet_b0', pretrained=True)
        self.features = nn.Sequential(*list(self.base_model.children())[:-1])
        
        enet_out_size = 1280 
        #make a classifier
        self.classifier = nn.Linear(enet_out_size, num_classes)
        
        
        
    def forward(self,x):
        # connect these parts and return the output
        x = self.features(x)
        output = self.classifier(x)
        return output
    
    

In [ ]:
model = SimpleCardClassifier(num_classes=53)

In [ ]:
print(str(model)[:500])

In [ ]:
example_out = model(images)
example_out.shape  ## [batch size, num classes]

# Step 3 : Training loop

* terms: 
    * Epoch : one run through the entire training dataset
    * step: one batch of data as defined in our dataloader
* This loop is one you will become familiar with when training models, you load in data to the model in batches - then calculate the loss and perform backpropagation.  There are packages that package this for you, but its good to have at least written it once to understand how itworks.

* Two things to select:
    * optimizer, adam is the best place to start for most tasks
    * loss function : what the model will optimize for

In [ ]:
# loss function
criterion = nn.CrossEntropyLoss()

# Optimizer function
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
criterion(example_out, labels
         )

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
num_epochs=5
train_loss, val_loss = [], []

model = SimpleCardClassifier(num_classes=53)
model.to(device)

for epoch in range(num_epochs):
    # set the model to train
    model.train()
    running_loss = 0.0 
    for images, labels in tqdm(train_loader, desc='training loop'):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * images.size(0)
    train_los =  running_loss / len(train_loader.dataset)
    train_loss.append(train_los)
    
    # validation phase
    model.eval()
    running_loss = 0.0 
    with torch.no_grad():
        for images, labels in tqdm(valid_loader, desc='validation loop'):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)
        val_los = running_loss / len(valid_loader.dataset)
        val_loss.append(val_los)
        
    print(f'Epoch {epoch+1}/{num_epochs} - Train loss: {train_los}, validationloss : {val_los}')